In [1]:
from tool import processor, correlation, resampling, score

### 1. Rank Clustering
- Open score files
- IQR example
- Compute the rankings of pre-tokenizers

In [2]:
path = './data/segment_level/raw'
f = processor.FileProcessor(path)

In [3]:
df = f.makeFrame()
df.head(3)

,sacreTer.Hannanum,sacreTer.Khaiii,sacreTer.Kiwi,sacreTer.Kkma,sacreTer.Mecab,sacreTer.Komoran,sacreTer.None,sacreTer.Okt,sacreTer.Spm,sacreBleu.Jamo,...,sacreChrf.Kkma,sacreChrf.Komoran,sacreChrf.Mecab,sacreChrf.None,sacreChrf.Okt,sacreChrf.Spm,sacreChrf.Character,sacreTer.Jamo,sacreTer.Character,Human
0,64.7059,50.0,50.0000,47.8261,50.0000,50.0000,77.7778,66.6667,50.0000,53.4154,...,53.3701,53.0081,52.3972,50.6958,48.1593,52.3972,54.0168,34.8485,44.8276,-1.0107
1,56.5217,62.5,54.1667,50.0000,60.0000,54.1667,66.6667,55.5556,59.0909,40.8603,...,44.6038,44.7165,41.2800,39.2493,41.2018,41.4562,41.8010,47.0588,52.9412,-1.0107
2,27.5862,25.0,25.0000,29.0323,25.8065,22.5806,45.4545,33.3333,25.8065,60.0980,...,66.3012,70.0629,68.7973,65.4657,67.0995,68.7973,67.6693,24.4444,25.6410,1.1051


In [4]:
# IQR
test_scores = df.iloc[:, 0].tolist()
s = score.Score(test_scores)
len(test_scores), len(s.iqr())

(7727, 7636)

In [5]:
ci, _mean = s.estimate_ci()
print('Confidence 95% = {:.2f} ~ {:.2f}'.format(_mean - ci, _mean + ci))

Confidence 95% = 34.43 ~ 103.18


In [4]:
# clustering
for metric in ['bleu', 'chrf', 'ter']:
    pearson = correlation.Correlation(f.selectFrame(metric))
    print(pearson.rank_cluster(), '\n')

[('sacreBleu.Character', 0), ('sacreBleu.Kiwi', 0), ('sacreBleu.Komoran', 0), ('sacreBleu.Jamo', 0), ('sacreBleu.Khaiii', 1), ('sacreBleu.Kkma', 2), ('sacreBleu.Mecab', 2), ('sacreBleu.Spm', 3), ('sacreBleu.Okt', 4), ('sacreBleu.Hannanum', 5), ('sacreBleu.None', 6)] 

[('sacreChrf.Komoran', 0), ('sacreChrf.Khaiii', 0), ('sacreChrf.Kiwi', 1), ('sacreChrf.Kkma', 2), ('sacreChrf.Character', 2), ('sacreChrf.Mecab', 2), ('sacreChrf.Spm', 2), ('sacreChrf.Okt', 3), ('sacreChrf.Hannanum', 4), ('sacreChrf.None', 5), ('sacreChrf.Jamo', 6)] 

[('sacreTer.None', 0), ('sacreTer.Hannanum', 1), ('sacreTer.Jamo', 1), ('sacreTer.Okt', 2), ('sacreTer.Character', 3), ('sacreTer.Khaiii', 4), ('sacreTer.Spm', 4), ('sacreTer.Komoran', 4), ('sacreTer.Mecab', 5), ('sacreTer.Kiwi', 6), ('sacreTer.Kkma', 6)] 



### 2. Bootstrap Resampling
- Wilcoxon rank sum test with variable `m, n, iteration`

In [11]:
# i.e = BLEU
samples = resampling.Resampling(frame=f.selectFrame('bleu'),
                    m=3000, n=1000, iteration=10)
samples.bootstrap_resampling(2)

,sacreBleu.Jamo,sacreBleu.Kiwi,sacreBleu.Kkma,sacreBleu.Komoran,sacreBleu.Hannanum,sacreBleu.Khaiii,sacreBleu.Mecab,sacreBleu.None,sacreBleu.Okt,sacreBleu.Spm,sacreBleu.Character,Human
2,1,1,0.5,0.5,3,1,1.5,5,3.5,1,0,0
4,0.75,1,1,1,3.5,1,1.25,5,3,1.5,0.25,0
6,1,1,1.167,1,3.667,0.833,1,5,2.833,1.333,0.5,0
8,1.25,0.75,0.875,0.875,3.75,0.75,0.75,5,2.75,1.375,0.75,0
10,1.2,0.7,0.8,0.9,3.7,0.8,0.7,5,2.9,1.3,0.6,0


### 3. Tokenizer

In [4]:
# Pre-tokenization = None
hypothesis = "모델 레온 데임(20)은 아무도 밟지 않은 것처럼 캣워크를 활보하며 \
    한 발을 다른 발 앞에 강하게 내딛고 활주로를 밟으며 강렬하고 강철 같은 섬광을 그대로 유지했다."
reference = ["모델 레옹 데임(20)은 아직 그 누구도 시도한 적 없는 방식으로 캣워크를 활보했다.\
    그는 강렬하고 찌를 듯한 눈빛을 유지한 채 한 발 한 발 힘차게 내뻗으며 런웨이를 밟아 나갔다."]

In [5]:
from tool.tokenizer import koNLPy, kakao, subword

In [3]:
# KoNLPy
komoran = koNLPy.KoNLPy('komoran')
print(komoran.tokenize(hypothesis))

['모델', '레온', '데', '임', '(', '20', ')', '은', '아무도', '밟', '지', '않', '은', '것', '처럼', '캐', '엇', '워크', '를', '활보', '하', '며', '한', '발', '을', '다른', '발', '앞', '에', '강하', '게', '내딛', '고', '활주로', '를', '밟', '으며', '강렬', '하', '고', '강철', '같', '은', '섬광', '을', '그대로', '유지', '하', '았', '다', '.']


In [4]:
# Kiwi
kiwi = kakao.Kiwi()
print(kiwi.tokenize(hypothesis))

['모델', '레온', '데이', 'ᆷ', '(', '20', ')', '은', '아무', '도', '밟', '지', '않', '은', '것', '처럼', '캣워크', '를', '활보', '하', '며', '한', '발', '을', '다른', '발', '앞', '에', '강하', '게', '내딛', '고', '활주로', '를', '밟', '으며', '강렬', '하', '고', '강철', '같', '은', '섬광', '을', '그대로', '유지', '하', '었', '다', '.']


In [5]:
tokens = kiwi.tokenize(hypothesis)
kiwi.token_to_sent(tokens)

'모델 레온 데이 ᆷ ( 20 ) 은 아무 도 밟 지 않 은 것 처럼 캣워크 를 활보 하 며 한 발 을 다른 발 앞 에 강하 게 내딛 고 활주로 를 밟 으며 강렬 하 고 강철 같 은 섬광 을 그대로 유지 하 었 다 .'

In [6]:
# Khaiii
khaiii = kakao.Khaiii()
print(khaiii.tokenize(hypothesis))

['모델', '레온', '데임', '(', '20', ')', '은', '아무', '도', '밟', '지', '않', '은', '것', '처럼', '캣워크', '를', '활보', '하', '며', '한', '발', '을', '다른', '발', '앞', '에', '강하', '게', '내딛', '고', '활주로', '를', '밟', '으며', '강렬', '하', '고', '강철', '같', '은', '섬광', '을', '그대로', '유지', '하', '였', '다', '.']


### 4. Automatic Metric Scoring 
- SacreBLEU: BLEU, TER, CHRF
- NLTK: BLEU, NIST, GLEU, RIBES, ...
- EED
- CharacTER

#### SacreBLEU

In [2]:
from tool.metric.sacreBLEU import SacreBLEU

In [3]:
metric = SacreBLEU('bleu')

metric.hypothesis=hypothesis
metric.references=reference

In [4]:
metric.get_score(sys_level=False)

BLEU = 12.35 37.9/14.3/11.1/7.7 (BP = 0.842 ratio = 0.853 hyp_len = 29 ref_len = 34)

In [5]:
metric.run_all()

TypeError: setattr expected 3 arguments, got 2

#### NLTK